In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import L2
import xgboost as xgb
from sklearn.metrics import accuracy_score
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**Distributed Denial of Service:** it's a cybersecurity menace which disrupts online services by sending an overwhelming amount of network traffic. These attacks are manually started with botnets that flood the target network. These attacks could have either of the following characteristics:
* The botnet sends a massive number of requests to the hosting servers.
* The botnet sends a high volume of random data packets, thus incapacitating the network.

These descriptions provide details about various attributes present in the dataset:
* **ip.src:** Source IP address
* **tcp.srcport:** Source port number for TCP (Transmission Control Protocol).
* **tcp.dstport:** Destination port number for TCP.
* **ip.proto:** IP protocol used (e.g., TCP, UDP).
* **frame.len:** Length of the network frame.
* **tcp.flags.syn:** TCP SYN flag.
* **tcp.flags.reset:** TCP RST flag.
* **tcp.flags.push:** TCP PUSH flag.
* **tcp.flags.ack:** TCP ACK flag.
* **ip.flags.mf:** IP More Fragments flag.
* **ip.flags.df:** IP Do Not Fragment flag.
* **ip.flags.rb:** Reserved bits in the IP header.
* **tcp.seq:** TCP sequence number.
* **tcp.ack:** TCP acknowledgment number.
* **frame.time:** Timestamp of the network frame.
* **Packets:** Number of packets in the network frame.
* **Bytes:** Number of bytes in the network frame.
* **Tx Packets:** Number of transmitted packets.
* **Tx Bytes:** Number of transmitted bytes.
* **Rx Packets:** Number of received packets.
* **Rx Bytes:** Number of received bytes.
* **Label:** The label or category assigned to the network event (e.g., 'DDoS-PSH-ACK', 'Benign', 'DDoS-ACK').

# Data Exploration & Cleaning:

In [ ]:
ddos_data=pd.read_csv('/kaggle/input/apaddos-dataset/APA-DDoS-Dataset/APA-DDoS-Dataset.csv')
ddos_data

In [ ]:
ddos_data.columns

In [ ]:
ddos_data.info()

In [ ]:
ddos_data.describe()

In [ ]:
label_counts=ddos_data['Label'].value_counts()
label_counts

In [ ]:
label_counts=ddos_data['Label'].value_counts()
plt.figure(figsize=(3, 3))
plt.pie(label_counts, labels=label_counts.index, autopct='%1.1f%%', startangle=140)
plt.title('Distribution of DDoS Attack Types')
plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
plt.show()

**Benign:** This label indicates that the network event or traffic is considered normal and does not pose any threat. In other words, it represents benign or legitimate network activity.

**DDoS-PSH-ACK:** This label represents a specific type of DDoS attack characterized by the TCP flags PSH (Push) and ACK (Acknowledgment). In this type of attack, the attacker sends a high volume of TCP packets with the PSH and ACK flags set, overwhelming the target server or network with a large number of connection requests.

**DDoS-ACK:** This label indicates another type of DDoS attack where the attacker floods the target server or network with a massive number of TCP packets with only the ACK (Acknowledgment) flag set. This flood of ACK packets consumes resources on the target system, leading to service disruption or denial of service for legitimate users.

The **PSH flag** indicates that the data should be pushed immediately to the application layer, which can cause the receiving system to process data more rapidly, potentially overwhelming its resources.

In [ ]:
ddos_data['ip.dst'].value_counts()

Since, we're only working on one ip.dst we can drop this column.

In [ ]:
ddos_data.drop('ip.dst',axis=1,inplace=True)

In [ ]:
ddos_data

It looks like we're analysing ddos attacks performed on only one ip address destination.

In [ ]:
ddos_data['ip.src'].value_counts()

These IP addresses represent the botnets utilized for the DDoS attacks. The consistent count of occurrences (10800) for each IP address implies an even distribution of DDoS attack traffic across various botnets.

In [ ]:
ddos_data['tcp.srcport'].value_counts()

In [ ]:
constant_features = ddos_data.columns[ddos_data.nunique() == 1]
constant_features

In [ ]:
print(f'tcp.dstport:{ddos_data["tcp.dstport"].value_counts()}')
print(f'ip.proto:{ddos_data["ip.proto"].value_counts()}')
print(f'tcp.flags.syn:{ddos_data["tcp.flags.syn"].value_counts()}')
print(f'tcp.flags.reset:{ddos_data["tcp.flags.reset"].value_counts()}')
print(f'tcp.flags.ack:{ddos_data["tcp.flags.ack"].value_counts()}')
print(f'ip.flags.mf:{ddos_data["ip.flags.mf"].value_counts()}')
print(f'ip.flags.rb:{ddos_data["ip.flags.rb"].value_counts()}')
print(f'tcp.seq:{ddos_data["tcp.seq"].value_counts()}')
print(f'tcp.ack:{ddos_data["tcp.ack"].value_counts()}')

It's better to drop constant features:

In [ ]:
ddos_data.drop(columns=constant_features,inplace=True)

In [ ]:
ddos_data.columns

In [ ]:
ddos_data['frame.time'].value_counts()

## Normalize `frame.time` column:

In [ ]:
ddos_data['frame.time'][0]

In [ ]:
def normalize_time_column():
    ddos_data['frame.time'] = ddos_data['frame.time'].str.replace(' Mountain Daylight Time', '')
    ddos_data['frame.time']=ddos_data['frame.time'].apply(lambda x: re.sub(r'\..*', '',x ))
    ddos_data['frame.time'] = pd.to_datetime(ddos_data['frame.time'], format=' %d-%b %Y %H:%M:%S')
    ddos_data['frame.time'] = ddos_data['frame.time'].dt.tz_localize('UTC').dt.tz_convert('US/Mountain')

In [ ]:
normalize_time_column()

In [ ]:
connection_by_time=ddos_data[ddos_data['Label']!='Benign'].groupby(['frame.time']).size()
connection_by_time

In [ ]:
ddos_data.dtypes

In [ ]:
connection_by_time_b=ddos_data[ddos_data['Label']=='Benign'].groupby(['frame.time']).size()
connection_by_time_b

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(connection_by_time.index, connection_by_time.values, marker='o', linestyle='-',label='Non-Benign')
plt.plot(connection_by_time_b.index, connection_by_time_b.values, marker='o', linestyle='-', label='Benign')
plt.title('Benign & non-Benign DDoS Connections Over Time')
plt.xlabel('Time')
plt.ylabel('Count')
plt.xticks(rotation=90)
plt.grid(True)
plt.tight_layout()
plt.legend()
plt.show()


In [ ]:
connection_by_time_different_ip_src=ddos_data[ddos_data['Label']!='Benign'].groupby(['ip.src','frame.time']).size()
connection_by_time_different_ip_src

In [ ]:
plt.figure(figsize=(10, 6))
connection_by_time_different_ip_src.unstack(level=0).plot(kind='bar', stacked=True)
plt.title('Count of Non-Benign DDoS Attacks by IP Source and Time')
plt.xlabel('Frame Time')
plt.ylabel('Count')
plt.legend(title='IP Source')
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()

In [ ]:
ddos_data[ddos_data['Label']=='Benign'].groupby(['frame.time','Packets']).size()

In [ ]:
non_benign_data = ddos_data[ddos_data['Label'] != 'Benign']
non_benign_bytes_by_time = non_benign_data.groupby(['frame.time', 'Bytes']).size().reset_index(name='count')
non_benign_bytes_by_time = non_benign_bytes_by_time.groupby('frame.time')['Bytes'].mean()

benign_data = ddos_data[ddos_data['Label'] == 'Benign']

benign_bytes_by_time =benign_data.groupby(['frame.time', 'Bytes']).size().reset_index(name='count')
benign_bytes_by_time = benign_bytes_by_time.groupby('frame.time')['Bytes'].mean()

plt.figure(figsize=(12, 6))
plt.plot(non_benign_bytes_by_time.index, non_benign_bytes_by_time.values, label='Non-Benign', linestyle='-', marker='o')
plt.plot(benign_bytes_by_time.index, benign_bytes_by_time.values, label='Benign', linestyle='-', marker='o')
plt.title('Average Bytes Received Over Time for non benign connections')
plt.xlabel('Time')
plt.ylabel('Average Bytes')
plt.xticks(rotation=90)
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

The average of bytes sent by timestamp varies for non-benign connections. But it's quite constant for benign connections.

In [ ]:
benign=ddos_data[ddos_data['Label']=="Bengin"].Packets
ddos_ack=ddos_data[ddos_data['Label']=="DDoS-ACK"].Packets
ddos_psh_ack=ddos_data[ddos_data['Label']=="DDoS-PSH-ACK"].Packets
plt.xlabel("Numbre of Packets")
plt.ylabel("")
plt.title("Numbre of packets for label ddos Visualiztion")
plt.hist([benign, ddos_ack,ddos_psh_ack], rwidth=0.95, color=['green','blue','red'],label=['Bengin','ddos_ack','ddos_psh_ack'])
plt.legend()

In [ ]:
# Filter non-benign and benign data
non_benign_data = ddos_data[ddos_data['Label'] != 'Benign']
benign_data = ddos_data[ddos_data['Label'] == 'Benign']

# Group by 'frame.time' and 'Packets' and calculate the mean for non-benign traffic
non_benign_packets_by_time = non_benign_data.groupby(['frame.time', 'Packets']).size().reset_index(name='count')
non_benign_packets_by_time = non_benign_packets_by_time.groupby('frame.time')['Packets'].mean()

# Group by 'frame.time' and 'Packets' and calculate the mean for benign traffic
benign_packets_by_time = benign_data.groupby(['frame.time', 'Packets']).size().reset_index(name='count')
benign_packets_by_time = benign_packets_by_time.groupby('frame.time')['Packets'].mean()

# Plotting the line chart for non-benign and benign traffic
plt.figure(figsize=(12, 6))
plt.plot(non_benign_packets_by_time.index, non_benign_packets_by_time.values, label='Non-Benign', linestyle='-', marker='o')
plt.plot(benign_packets_by_time.index, benign_packets_by_time.values, label='Benign', linestyle='-', marker='o')
plt.title('Average Packets Received Over Time')
plt.xlabel('Time')
plt.ylabel('Average Packets')
plt.xticks(rotation=90)
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

While the number of packets per frame for non-benign traffic varies, the number of packets per frame for benign traffic is constant = 10.

In [ ]:
# Filter non-benign and benign data
non_benign_data = ddos_data[ddos_data['Label'] != 'Benign']
benign_data = ddos_data[ddos_data['Label'] == 'Benign']

# Plot histograms for non-benign and benign connections
plt.figure(figsize=(12, 6))
plt.hist(non_benign_data['Packets'], bins=30, alpha=0.7, color='red', label='Non-Benign')
plt.hist(benign_data['Packets'], bins=30, alpha=0.7, color='blue', label='Benign')
plt.title('Distribution of Packets Over Time')
plt.xlabel('Number of Packets')
plt.ylabel('Frequency')
plt.legend()
#plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
label_counts = ddos_data.groupby(['frame.time','Label']).size().unstack(fill_value=0)
label_counts

In [ ]:
label_counts['Benign'].plot(kind='line', figsize=(10, 6))
label_counts['DDoS-PSH-ACK'].plot(kind='line', figsize=(10, 6))
label_counts['DDoS-ACK'].plot(kind='line', figsize=(10, 6))

plt.title('Frequency of Labels Over Time')
plt.xlabel('Time')
plt.xticks(rotation=90)
plt.ylabel('Count')
plt.legend()
plt.show()

# Correlation Analysis:

In [ ]:
ddos_data.hist(bins =10, figsize=(20, 15))
plt.show()

In [ ]:
sns.pairplot(ddos_data,hue='Label',vars=['frame.time','Packets','Tx Packets'])

In [ ]:
numerical_features = [feature for feature in ddos_data.columns if ddos_data[feature].dtypes != 'O']
numerical_features

In [ ]:
categorical_features = [feature for feature in ddos_data.columns if ddos_data[feature].dtypes == 'O']
categorical_features

In [ ]:
corr_matrix=ddos_data[numerical_features].corr()
corr_matrix

In [ ]:
plt.figure(figsize=(12, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=.5)
plt.title('Correlation Heatmap of Numerical Features')
plt.show()

In [ ]:
#Let's examine binary columns
binary_features=ddos_data.columns[ddos_data.nunique() == 2]
binary_features

We don't want to normalize binary features.

In [ ]:
numerical_features.remove('Tx Bytes')
numerical_features.remove('Rx Bytes')
ddos_data.drop(['Tx Bytes','Rx Bytes'],axis=1, inplace=True)
numerical_features = [feature for feature in numerical_features if feature not in binary_features]

In [ ]:
numerical_features 

In [ ]:
def z_score_normalization(x):
    return (x - x.mean()) / x.std()

In [ ]:
ddos_data[numerical_features ] = ddos_data[numerical_features ].apply(z_score_normalization)
ddos_data[numerical_features ].isnull().sum()

## Labeling Catgorical data:

If we were to use sigmoid as the final activation of our neural network we have to encode the Label column using LabelEncoder.
If we're using softmax a.k.a classifying into 3 classes, we use One Hot encoder.

In [ ]:
label_encoder=LabelEncoder()
def Encode_ip_src_column():
    ddos_data['ip.src']=label_encoder.fit_transform(ddos_data['ip.src'])

In [ ]:
def Encode_Label_Column_1():
    label_mapping = {'Benign': 0, 'DDoS-PSH-ACK': 1, 'DDoS-ACK': 1}
    ddos_data['label_encoded']=label_encoder.fit_transform(ddos_data['Label'])
    ddos_data['label_encoded'] = ddos_data['Label'].map(label_mapping)

In [ ]:
Encode_ip_src_column()
Encode_Label_Column_1()

In [ ]:
ddos_data['label_encoded'].value_counts()

In [ ]:
ddos_data['ip.src'].value_counts()

In [ ]:
def Encode_Label_Column_2():
    data1=pd.get_dummies(ddos_data,columns=['Label'],drop_first=False,dtype='int64')
    return data1

In [ ]:
ddos_data=Encode_Label_Column_2()

In [ ]:
ddos_data

# Split train, test data:

In [ ]:
y=ddos_data['label_encoded']
X=ddos_data.drop(columns=['label_encoded','Label_Benign','Label_DDoS-ACK','Label_DDoS-PSH-ACK'])

In [ ]:
X.columns

In [ ]:
X.head()

In [ ]:
X_train_val, X_test, y_train_val, y_test=train_test_split(X,y,test_size=0.1,random_state=42)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.125, random_state=42)

In [ ]:
len(X_train.columns)

The distribution of classes between the two train and test sets is even.

# Defining the deep learning model:

In [ ]:
model=Sequential([
    Input(shape=(10,)),
    Dense(units=5,activation='relu',name='Hidden_layer_1',kernel_regularizer=L2(0.3)),
    #Dense(units=2,activation='relu',name='Hidden_layer_2',kernel_regularizer=L2(0.3)),
    Dense(units=1,activation='sigmoid',name='Output_layer',kernel_regularizer=L2(0.1))
])
model.compile(optimizer=Adam(learning_rate=1e-3),loss=BinaryCrossentropy(),metrics=['accuracy'])
model.summary()

## Fitting the model:

In [ ]:
history_log = model.fit(
    X_train, 
    y_train, 
    batch_size=1024, 
    epochs=100, verbose=2, 
    callbacks=None, 
    shuffle=True,
    validation_data=(X_val,y_val),  
    class_weight=None, 
    sample_weight=None, 
    initial_epoch=0)

## Plotting the loss by epochs:

In [ ]:
loss = history_log.history['loss']
val_loss = history_log.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'g', label = 'Training Loss')
plt.plot(epochs, val_loss, 'r', label = 'Validation Loss')
plt.title('Loss v/s No. of epochs')
plt.xlabel('Number of Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

## Plotting the accuracy by epochs:

In [ ]:
accuracy = history_log.history['accuracy']
val_accuracy = history_log.history['val_accuracy']
plt.plot(epochs, accuracy , 'g', label = 'Training accuracy')
plt.plot(epochs, val_accuracy , 'r', label = 'Validation accuracy')
plt.title('Accuracy Scores v/s Number of Epochs')
plt.xlabel('No. of Epochs')
plt.ylabel('Accuracy Score')
plt.legend()
plt.show()

## Model evaluation for test set:

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)
print('Accuracy of Deep neural Network on unseen data : %.2f' % (accuracy*100))